#Kütüphaneleri ve veri setini yükleme

In [1]:
from sklearn import datasets
import pandas as pd
import math
import numpy as np


In [8]:
#data=pd.read_csv("data2.csv")
iris=datasets.load_iris()

Veri Önişleme

In [10]:
x=iris.data
y=iris.target

In [35]:
xp=pd.DataFrame(x,columns=["sepal_length","sepal_width","petal_length","pedal_width"])

In [36]:
yp=pd.DataFrame(y,columns=["Class"])

In [108]:
yp.head()

,Class
0,0
1,0
2,0
3,0
4,0


In [40]:
xp.head()

,sepal_length,sepal_width,petal_length,pedal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [42]:
xp.iloc[:5,0]

0    5.1
1    4.9
2    4.7
3    4.6
4    5.0
Name: sepal_length, dtype: float64

#Verideki tüm nitelikleri 3 kategoriye ayırıyoruz (0,1,2)

In [45]:
max(xp.iloc[:,0])

7.9

In [109]:
maksimumlar=xp.max() #hepsinin maximumu

In [110]:
minimumlar=xp.min()

In [111]:
birimparca=(maksimumlar-minimumlar)/3
print(birimparca)
ilksinir=minimumlar+birimparca
print(ilksinir)
ikincisinir=maksimumlar-birimparca
print(ikincisinir)

sepal_length       1.200000
sepal_width        0.800000
petal_length       1.966667
pedal_width        0.800000
sepal_kategorik    0.666667
dtype: float64
sepal_length       5.500000
sepal_width        2.800000
petal_length       2.966667
pedal_width        0.900000
sepal_kategorik    0.666667
dtype: float64
sepal_length       6.700000
sepal_width        3.600000
petal_length       4.933333
pedal_width        1.700000
sepal_kategorik    1.333333
dtype: float64


In [117]:
kategorik=pd.DataFrame()
labels = [0,1,2]

for i in range(4):
  bins = [-np.inf,ilksinir[i], ikincisinir[i], np.inf]    
  temp = pd.cut(xp.iloc[:,i], bins=bins, labels=labels)
  kategorik[xp.columns[i]+"_ktgri"]=temp
kategorik.head()



,sepal_length_ktgri,sepal_width_ktgri,petal_length_ktgri,pedal_width_ktgri
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


In [118]:
data=kategorik
data["Class"]=yp
data.head()

,sepal_length_ktgri,sepal_width_ktgri,petal_length_ktgri,pedal_width_ktgri,Class
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0


#entropi fonksiyonu

In [115]:
def compute_impurity(feature, impurity_criterion):
    probs = feature.value_counts(normalize=True) #nitelik türünün oranı örn: n_yas/n_toplam

    if impurity_criterion == 'entropy':
        impurity = -1 * np.sum(np.log2(probs) * probs)
    elif impurity_criterion == 'gini':
        impurity = 1 - np.sum(np.square(probs))
    else:
        raise ValueError('Unknown impurity criterion')
        
    return(round(impurity, 3))



#Information Gain ve Gain Ratio Fonksiyonu

In [116]:
def information_gain(df, target, descriptive_feature, split_criterion):
    
    print('target feature:', target) #hedef/sonuç sınıf
    print('descriptive_feature:', descriptive_feature) #incelenen sınıf
    print('split criterion:', split_criterion) #bölme yöntemi (gini,entropy)
            
    target_entropy = compute_impurity(df[target], split_criterion) #InfoD/GiniD hesabı
    
    entropy_list = list()
    weight_list = list()
    
    
   
    for level in df[descriptive_feature].unique(): #niteliğin her kategorisi için
        df_feature_level = df[df[descriptive_feature] == level] #ilgili alt kategori için filtrele       
        entropy_level = compute_impurity(df_feature_level[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level) / len(df)
        weight_list.append(round(weight_level, 3))

    feature_split_info = compute_impurity(df[descriptive_feature], "entropy")
    print("Split_Info_",descriptive_feature,":",feature_split_info)
    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))   
    print('InfoD_',descriptive_feature, feature_remaining_impurity)
    
    information_gain = target_entropy - feature_remaining_impurity
    print('information gain:', information_gain)

    feature_gain_ratio=information_gain/feature_split_info
    print("gain ratio: ", feature_gain_ratio)


    
    print('====================')

    return(information_gain)

#Information Gain ve Gain Ratio Hesaplıyoruz

In [119]:
#Class : çıktı sınıfı
split_criterion = 'entropy' #gini/entropy
for feature in data.drop(columns='Class').columns: 
  feature_info_gain = information_gain(data, 'Class', feature, split_criterion)

target feature: Class
descriptive_feature: sepal_length_ktgri
split criterion: entropy
Split_Info_ sepal_length_ktgri : 1.428
InfoD_ sepal_length_ktgri 0.972023
information gain: 0.612977
gain ratio:  0.4292556022408964
target feature: Class
descriptive_feature: sepal_width_ktgri
split criterion: entropy
Split_Info_ sepal_width_ktgri : 1.308
InfoD_ sepal_width_ktgri 1.3191849999999998
information gain: 0.26581500000000013
gain ratio:  0.20322247706422028
target feature: Class
descriptive_feature: petal_length_ktgri
split criterion: entropy
Split_Info_ petal_length_ktgri : 1.582
InfoD_ petal_length_ktgri 0.260286
information gain: 1.324714
gain ratio:  0.8373666245259165
target feature: Class
descriptive_feature: pedal_width_ktgri
split criterion: entropy
Split_Info_ pedal_width_ktgri : 1.582
InfoD_ pedal_width_ktgri 0.206557
information gain: 1.3784429999999999
gain ratio:  0.8713293299620732


#Gini Indeks Fonksiyonu

In [120]:
def gini_Index(df, target, descriptive_feature, split_criterion):
    
    print('target feature:', target) #hedef/sonuç sınıf
    print('descriptive_feature:', descriptive_feature) #incelenen sınıf
    print('split criterion:', split_criterion) #bölme yöntemi (gini,entropy)
            
    target_entropy = compute_impurity(df[target], split_criterion) #InfoD/GiniD hesabı
  
    entropy_list = list()
    weight_list = list()
    
    # loop over each level of the descriptive feature
    # to partition the dataset with respect to that level
    # and compute the entropy and the weight of the level's partition
    levels=data[feature].unique()
    uzunluk=len(levels)
    if uzunluk==2:
      for level in levels: #niteliğin her kategorisi için
          df_feature_level = df[df[descriptive_feature] == level] #ilgili alt kategori için filtrele       
          entropy_level = compute_impurity(df_feature_level[target], split_criterion)
          entropy_list.append(round(entropy_level, 3))
          weight_level = len(df_feature_level) / len(df)
          weight_list.append(round(weight_level, 3))
      feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
      print('InfoD_',descriptive_feature, feature_remaining_impurity)
      
      information_gain = target_entropy - feature_remaining_impurity
      print('information gain:', information_gain)
      
      print('====================')

     
    elif uzunluk==3:
      indisler=[[0,1,2],[1,0,2],[2,0,1]]
      for ind in indisler:
        entropy_list = list()
        weight_list = list()
        birincil=levels[ind[0]]
        df_feature_level1 = df[df[descriptive_feature] == birincil]     
        entropy_level = compute_impurity(df_feature_level1[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level1) / len(df)
        weight_list.append(round(weight_level, 3))
        ikincil1=levels[ind[1]]
        ikincil2=levels[ind[2]]
        df_feature_level2 = df[df[descriptive_feature] == ikincil1]
        df_feature_level3 = df[df[descriptive_feature] == ikincil2]
        df_concatted=pd.concat([df_feature_level2,df_feature_level3])
        entropy_level = compute_impurity(df_concatted[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_concatted) / len(df)
        weight_list.append(round(weight_level, 3))

        print("Gini_",birincil,", {",ikincil1,", ",ikincil2,"}")

        feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
        print('InfoD_',descriptive_feature, feature_remaining_impurity)
        
        information_gain = target_entropy - feature_remaining_impurity
        print('information gain:', information_gain)
        
        print('====================')

       
    
    else:
      print("tanımsız")


  

# Gini Index Hesabı

In [121]:
#Class : çıktı sınıfı
split_criterion = 'gini' #gini/entropy
for feature in data.drop(columns='Class').columns: 
  feature_info_gain = gini_Index(data, 'Class', feature, split_criterion)

target feature: Class
descriptive_feature: sepal_length_ktgri
split criterion: gini
Gini_ 0 , { 1 ,  2 }
InfoD_ sepal_length_ktgri 0.448365
information gain: 0.21863500000000002
Gini_ 1 , { 0 ,  2 }
InfoD_ sepal_length_ktgri 0.551175
information gain: 0.11582500000000007
Gini_ 2 , { 0 ,  1 }
InfoD_ sepal_length_ktgri 0.603534
information gain: 0.06346600000000002
target feature: Class
descriptive_feature: sepal_width_ktgri
split criterion: gini
Gini_ 1 , { 2 ,  0 }
InfoD_ sepal_width_ktgri 0.651457
information gain: 0.015543000000000085
Gini_ 2 , { 1 ,  0 }
InfoD_ sepal_width_ktgri 0.6180000000000001
information gain: 0.04899999999999993
Gini_ 0 , { 1 ,  2 }
InfoD_ sepal_width_ktgri 0.593249
information gain: 0.07375100000000001
target feature: Class
descriptive_feature: petal_length_ktgri
split criterion: gini
Gini_ 0 , { 1 ,  2 }
InfoD_ petal_length_ktgri 0.3335
information gain: 0.3335
Gini_ 1 , { 0 ,  2 }
InfoD_ petal_length_ktgri 0.40280000000000005
information gain: 0.2642
Gini_ 